# Dgl

> Dgl functions

In [ ]:
#| default_exp dgl

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from ast import Call
import itertools, math, numpy as np, pandas as pd
import dgl, dgl.function as fn

import torch, torch.nn as nn, torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torchdiffeq

from typing import Callable, Union, Literal

# NOTE: could try to make dynamic decorator that wraps over the function
# class DGLLayer:
#     def __init__(self, layer:Union[
#         dgl.nn.pytorch.conv.GraphConv,
#         dgl.nn.pytorch.conv.GATConv,
#         dgl.nn.pytorch.conv.GATv2Conv,
#         dgl.nn.pytorch.conv.TAGConv,
#         dgl.nn.pytorch.conv.SAGEConv,
#         dgl.nn.pytorch.conv.DOTGatConv,
#         dgl.nn.pytorch.conv.PNAConv,
#     ]):
#         self.graph = graph
#         self.layer = layer

#     def __call__(self, graph:dgl.DGLGraph, *args, **kwargs):
#         self.layer(*args, **kwargs)

# class DGLGATConv(dgl.nn.pytorch.conv.GATConv):
#     def __init__(self, graph:dgl.DGLGraph, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.g = graph
        
#     def forward(self, feat, get_attention=False):
#         return super().forward(self.g, feat, get_attention)
#dgl.nn.pytorch.conv.GraphConv,  in_feats, out_feats, norm='both', weight=True, bias=True, activation=None, allow_zero_in_degree=False
#dgl.nn.pytorch.conv.TAGConv,  in_feats, out_feats, k=2, bias=True, activation=None
#dgl.nn.pytorch.conv.GATConv,  (in_feats, out_feats, num_heads, feat_drop=0.0, attn_drop=0.0, negative_slope=0.2, residual=False, activation=None, allow_zero_in_degree=False, bias=True
#dgl.nn.pytorch.conv.GATv2Conv,  (in_feats, out_feats, num_heads, feat_drop=0.0, attn_drop=0.0, negative_slope=0.2, residual=False, activation=None, allow_zero_in_degree=False, bias=True, share_weights=False)
#dgl.nn.pytorch.conv.SAGEConv,  in_feats, out_feats, aggregator_type, feat_drop=0.0, bias=True, norm=None, activation=None
#dgl.nn.pytorch.conv.PNAConv,  in_size, out_size, aggregators, scalers, delta, dropout=0.0, num_towers=1, edge_feat_size=0, residual=Tru
#dgl.nn.pytorch.conv.DotGatConv,  (in_feats, out_feats, num_heads, allow_zero_in_degree=False

In [ ]:
#| export
import dgl.nn.pytorch.conv as conv

class MeanAttentionLayer(nn.Module):
    def __init__(self, axis=1):
        super().__init__()
        self.axis = axis
    def forward(self, x):
        return torch.mean(x, axis=self.axis)

class SumAttentionLayer(nn.Module):
    def __init__(self, axis=1):
        super().__init__()
        self.axis = axis
    def forward(self, x):
        return torch.sum(x, axis=self.axis)

class DGLSequential(nn.Module):
    def __init__(self, graph, *args):
        super(DGLSequential, self).__init__()
        self.graph = graph
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, graph, input, **kwargs):
        for module in self:
            input = module(graph, input)
        return input


class DGLDotGatConv(conv.DotGatConv):
    def __init__(self, graph, in_feats, out_feats, num_heads, allow_zero_in_degree=False):
         super(DGLDotGatConv, self).__init__(in_feats, out_feats, num_heads, allow_zero_in_degree)
         self.graph = graph

    def forward(self, feat, get_attention=False):
        return super().forward(self.graph, feat, get_attention)

class DGLTAGConv(conv.TAGConv):
    def __init__(self, graph, in_feats, out_feats, k=2, bias=True, activation=None):
         super(DGLTAGConv, self).__init__(in_feats, out_feats, k=k, bias=bias, activation=activation)
         self.graph = graph

    def forward(self, feat, edge_weight=None):
        return super().forward(self.graph, feat, edge_weight)

class DGLGATConv(conv.GATConv):
    def __init__(
            self, graph, in_feats, out_feats, num_heads, 
            feat_drop=0.0, attn_drop=0.0, negative_slope=0.2, residual=False, 
            activation=None, allow_zero_in_degree=False, bias=True
        ):
         super(DGLGATConv, self).__init__(
             in_feats, out_feats, num_heads, feat_drop=feat_drop, attn_drop=attn_drop, 
             negative_slope=negative_slope, residual=residual, activation=activation, 
             allow_zero_in_degree=allow_zero_in_degree, bias=bias
            )
         self.graph = graph

    def forward(self, feat, get_attention=False):
        return super().forward(self.graph, feat, get_attention)

class DGLGATv2Conv(conv.GATv2Conv):
    def __init__(
            self, graph, in_feats, out_feats, num_heads, feat_drop=0.0, attn_drop=0.0, negative_slope=0.2, 
            residual=False, activation=None, allow_zero_in_degree=False, bias=True, share_weights=False
        ):
         super(DGLGATv2Conv, self).__init__(
             in_feats, out_feats, num_heads, feat_drop=feat_drop, attn_drop=attn_drop, 
             negative_slope=negative_slope, residual=residual, activation=activation, 
             allow_zero_in_degree=allow_zero_in_degree, bias=bias, share_weights=share_weights
            )
         self.graph = graph

    def forward(self, feat, get_attention=False):
        return super().forward(self.graph, feat, get_attention)

class DGLSAGEConv(conv.SAGEConv):
    def __init__(self, graph, in_feats, out_feats, aggregator_type, feat_drop=0.0, bias=True, norm=None, activation=None):
         super(DGLSAGEConv, self).__init__(
             in_feats, out_feats, aggregator_type, feat_drop=feat_drop, 
             bias=bias, norm=norm, activation=activation
            )
         self.graph = graph

    def forward(self, feat, edge_weight=None):
        return super().forward(self.graph, feat, edge_weight)


class DGLPNAConv(conv.PNAConv):
    def __init__(
            self, graph, in_size, out_size, aggregators, scalers, delta, 
            dropout=0.0, num_towers=1, edge_feat_size=0, residual=True
        ):
         super(DGLPNAConv, self).__init__(in_size, out_size, aggregators, scalers, delta, dropout=dropout, num_towers=num_towers, edge_feat_size=edge_feat_size, residual=residual)
         self.graph = graph

    def forward(self, node_feat, edge_feat=None):
        return super().forward(self.graph, node_feat, edge_feat)

: 

In [ ]:
#| export
from gode.utils import is_list_like
from dgl.nn.pytorch import GATConv

class GODE(nn.Module):
    def __init__(self, in_feats, out_feats, num_heads, activation=nn.Tanh):
        super(GODE, self).__init__()

        self.in_feats = in_feats
        self.out_feats = out_feats
        self.num_heads = num_heads
        
        num_heads = num_heads if is_list_like(num_heads) else [num_heads]
        
        
        
        for idx, module in enumerate([
            GATConv(
                in_feats=in_feats, out_feats=out_feats, 
                num_heads=n_heads, residual=False,
                activation=activation if i <= (len(num_heads) - 1) else None, 
                feat_drop=0.0, 
                attn_drop=0.0
            ) for i, n_heads in enumerate(num_heads)
        ]):
            self.add_module(str(idx), module)

        

    def forward(self, graph, feat, get_attention=False):
        attns = []
        for layer in self:
            if get_attention:
                feat, attn = layer(graph, feat, get_attention)
                attns.append(attn)
            else:
                feat = layer(graph, feat)
        
        if get_attention:
            return feat, attns
        return feat
        

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()